<a href="https://colab.research.google.com/github/vinayak-pathak/deepimaging.github.io/blob/master/BME_MML_Lecture_2_Tensorflow_Fundamentals_and_Python_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dependencies

In [ ]:
!pip3 install tensorflow_datasets
!pip3 install tensorflow-gpu==2.0.0-rc0

%load_ext tensorboard

!nvidia-smi

# Review decorators and with blocks

In [ ]:
class controlled_execution:
    def __enter__(self):
        print('setting things up (such as opening a file)')
        x = 1/0
    def __exit__(self, type, value, traceback): # What are the __underscores__ for?
        print(value)
        print(traceback)
        print(type)
        print('removing things (such as closing a file)')

with controlled_execution():
  x = 1

In [ ]:
class apple:
  def eat(self):
    print('yum')
  def __init__(self):
    self.value = 1
    print('created apple')
    
  def __add__(self, b):
    return self.value + b.value

class orange:
  def __init__(self):
    self.value = 1
    print('created orange')
    
  def __add__(self, b):
    return self.value + b.value

a = apple()
o = orange()

a.__add__(o)

created apple


In [ ]:
def foo(fn):
  def function_wrapper(x):
    print('Before I call my fn')
    fn(x)
    print('After I call my fn')
    
  return function_wrapper
  
def my_fun(x):
  return 2*(x+x)

function_wrapper()

In [ ]:
def my_decorator(fn):
  def function_wrapper(x):
    print("Before calling " + fn.__name__)
    output = fn(x)
    print("After calling " + fn.__name__)
    return output
  return function_wrapper

my_decorator(my_fun)(2)

@my_decorator
def my_fun(x):
  return 2*(x+x)

print(my_fun(2))

# Explore `@tf.function`
We will create the following graph in TensorFlow 2.0 using `@tf.function`

![alt text](https://miro.medium.com/max/1917/1*RfSvkiVUHHWEe805Hf1HEw.png)

In [ ]:
import tensorflow as tf

@tf.function
def my_func(my_input):
    print(tf.executing_eagerly())
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    return e,f

# We use `tf.summary` to trace the graph as it is executing.
tf.summary.trace_on(graph=True, profiler=False)

print(my_func(2.0))

with tf.summary.create_file_writer('./logs').as_default():   
    tf.summary.trace_export(
        name='tf2_graph',
        step=0,
        profiler_outdir='./logs/profiler'
    )
    
tf.summary.trace_off()

False
(<tf.Tensor: id=41, shape=(), dtype=float32, numpy=-1.0>, <tf.Tensor: id=42, shape=(), dtype=float32, numpy=nan>)


In [ ]:
import tensorflow as tf

@tf.function
def my_func(my_input):
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    return e,f

# We use `tf.summary` to trace the graph as it is executing.
tf.summary.trace_on(graph=True, profiler=False)
print(my_func(2.0))
with tf.summary.create_file_writer('./logs').as_default():   
    tf.summary.trace_export(
        name='tf2_graph',
        step=0,
        profiler_outdir='./logs/profiler'
    )
tf.summary.trace_off()

(<tf.Tensor: id=301, shape=(), dtype=float32, numpy=-1.0>, <tf.Tensor: id=302, shape=(), dtype=float32, numpy=nan>)


In [ ]:
# We can also get an explicit tf.Graph using this with block
graph = tf.Graph()
with graph.as_default():
  my_func(2.0)
graph.as_graph_def()

In [ ]:
# tf.Variable

@tf.function #(what happens if we try this?)
def f(const):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.) + const
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    return y

f(100)

PRINT:  Tensor("add_1:0", shape=(2, 2), dtype=float32)


ValueError: ignored

In [ ]:
f(100)

<tf.Tensor: id=150, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>

In [ ]:
# What is the difference between class and instance?

class F:
    def __init__(self):
        self._b = None

    @tf.function
    def __call__(self, const):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        if self._b is None:
            self._b = tf.Variable(12.)
        self._b.assign_add(const)
        y = tf.matmul(a, x) + self._b
        print("PRINT: ", y)
        tf.print("TF-PRINT: ", y)
        return y

f = F()
f(100.)

PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)
PRINT:  Tensor("add:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[122 122]
 [123 113]]


<tf.Tensor: id=362, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>

In [ ]:
def get_next_data(i):
  filebytes = open_file('{}.jpg'.format(i))
  array = decode_jpeg(filebytes)
  skewed_array = skew(array)
  return skewed_array

for i in range(steps):
  batch = get_next_data(i)
  train_on_gpu(batch)


In [ ]:
my_files = map(range(steps), lambda i: open_file('{}.jpg'.format(i)))
my_arrays = map(range(my_files), lambda filebytes: decode_jpeg(filebytes))
my_skewed_arrays = map(range(my_arrays), lambda array: skew(array))



TypeError: ignored

In [ ]:
def fun(val, *args,  keyword_one=None, keyword_two=None, **kwargs):
  print(args)
  print(kwargs)
  
fun(2,3,4,5,6,7,8,9, keyword_one='hi', keyword_two='hello', keyword_three='exists?', keyword_four='?')

(3, 4, 5, 6, 7, 8, 9)
{'keyword_three': 'exists?', 'keyword_four': '?'}
